5단계 까지 도출한 사항을 정리하자면,

Weight Capacity (kg)는 Price를 예측할 수 있는 Key feature 이다. Weight Capacity (kg)는 범주형 성격을 보이는데, 

이 배경에는 동일한 Weight Capacity (kg)에는 동일한 노이즈가 부여되었음이 짐작이 된다. 

이 점은 phase1~5 까지의 여러 실험을 통해 이로 인해 나타나는 결과를 통해 유력함을 확인했습니다. 

단계에서는 이 점을 이용한 Weight Capacity (kg)를  train에 등장하는 Weight Capacity (kg)와 5, 30을 제외한 변수는

Weight Capacity (kg)로 Mean encoding을 입력으로 하는 모델을 이용하고, 

그렇지 않으면 Mean Encoding을 하지 않은 모델을 사용합니다.

In [1]:
import os, sys

import pandas as pd
import polars as pl

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy

import sklearn
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import dproc, sgml, sgnn, sgpp, sgutil, custpp

print(sys.version)
for i in [pd, pl, mpl, sns, np, scipy, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        pass

2025-02-10 23:35:06.664344: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 23:35:06.908394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 23:35:07.676875: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


3.12.3 (main, May  1 2024, 17:33:23) [GCC 11.4.0]
pandas 2.2.2
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
numpy 1.26.4
scipy 1.13.0
sklearn 1.4.2
lightgbm 4.3.0
xgboost 2.1.2
catboost 1.2.5


In [2]:
from itertools import combinations

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, train_test_split
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [3]:
sc = sgutil.SGCache('img', 'result')
ss = ShuffleSplit(n_splits = 1, train_size = 0.8, random_state = 123)
kf = KFold(5, random_state = 123, shuffle=True)

files = {
    'train': os.path.join('data', 'train.csv'),
    'train_extra': os.path.join('data', 'training_extra.csv'),
    'test': os.path.join('data', 'test.csv'),
}

t = sc.cache_result(
    'pipeline_2',
    lambda : make_pipeline(
        sgpp.PolarsProcessor(), 
        sgpp.ExprProcessor({
            'Compartments_c' : pl.col('Compartments').cast(pl.String).cast(pl.Categorical)
        }),
        sgpp.PandasCoverter(index_col = 'id'),
        sgpp.ApplyWrapper(
            sgpp.CatArrangerFreq(1, na_value = 'Unknown'),
            ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
        ), 
        custpp.WeightCapacityProcessor()
    ).fit(files['train']),
    rerun = 1
)
df_train = pd.concat(
    [t.transform(files['train']), t.transform(files['train_extra'])], axis = 0
)
df_test = t.transform(files['test'])

target = 'Price'
X_cat = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Compartments_c']
X_num = ['Weight Capacity (kg)']

In [4]:
def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index).clip(15, 150),
    'score_func': lambda df, prds: root_mean_squared_error(df[target].sort_index(), prds.sort_index()),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

cb_adapter = sgml.CBAdapter(cb.CatBoostRegressor)
lr_adapter = sgml.SklearnAdapter(LinearRegression)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor)

In [6]:
def eval_model(hparams, adapter, case = 0, sp = ss, **args):
    results = list()
    for train_idx, valid_idx in sp.split(df_train, df_train[target]):
        df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
        bidx = df_valid['Weight Capacity (kg)'].isin(df_cv_train['Weight Capacity (kg)'].unique()) & df_valid['Weight Capacity (kg)'].notna() &\
                df_valid['Weight Capacity (kg)'].between(5, 30, inclusive = 'neither')
        if case == 1:
            bidx = ~bidx
        df_valid = df_valid.loc[bidx]
        reg = sgml.train(df_cv_train, hparams, config, adapter, **args)
        prd = make_pipeline(reg[0]['preprocessor'], reg[0]['model']).predict(df_valid[reg[1]])
        results.append(
            (reg, root_mean_squared_error(df_valid[target], prd), pd.Series(prd, index = df_valid.index, name = Target))
        )
    return results

In [17]:
results = sc.cache_result(
    'cv_cb1_0',
    lambda : eval_model({
        'model_params' : {'n_estimators': 700, 'max_depth': 7, 'learning_rate': 0.1},
        'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123},
        #'validation_fraction': 0.1
    }, cb_adapter, case = 0, sp = kf, task_type = 'GPU')
)
sgml.cb_learning_result(results[0][0][0])['valid_result'].idxmin(), results[0][1]

(metric  set  
 RMSE    learn    699
 dtype: int64,
 38.637942532234035)

In [16]:
results = sc.cache_result(
    'cv_cb1_1',
    lambda :eval_model({
        'model_params' : {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.1},
        'X_num': X_num, 'X_cat': X_cat,
    #    'validation_fraction': 0.1
    }, cb_adapter, case = 1, sp = kf, task_type = 'GPU')
)
sgml.cb_learning_result(results[0][0][0])['valid_result'].idxmin(), results[0][1]

(metric  set  
 RMSE    learn    999
 dtype: int64,
 39.22134932845147)